# <div align='center'> 测试新架构API </div>

In [1]:
import requests
import json

In [55]:
hostname = 'gamma'
host = '10.255.20.227'
port = 8129

user = 'test'
uuid = '123456'

## 机器学习框架

### 1. 训练`/k12ai/framework/train`

In [57]:
config = '''{
    "dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": true,
        "granularity": "5-class"
    },
    "validation_dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": false,
        "granularity": "5-class"
    },
    "train_data_path": "/data/datasets/nlp/sst/train.txt",
    "validation_data_path": "/data/datasets/nlp/sst/dev.txt",
    "test_data_path": "/data/datasets/nlp/sst/test.txt",
    "model": {
        "type": "bcn",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 50,
                    "pretrained_file": "/data/datasets/nlp/glove/glove.6B.50d.txt.gz",
                    "trainable": false
                }
            }
        },
        "embedding_dropout": 0.25,
        "pre_encode_feedforward": {
            "input_dim": 50,
            "num_layers": 1,
            "hidden_dims": [50],
            "activations": ["relu"],
            "dropout": [0.25]
        },
        "encoder": {
            "type": "lstm",
            "input_size": 50,
            "hidden_size": 50,
            "num_layers": 1,
            "bidirectional": true
        },
        "integrator": {
            "type": "lstm",
            "input_size": 300,
            "hidden_size": 50,
            "num_layers": 1,
            "bidirectional": true
        },
        "integrator_dropout": 0.1,
        "output_layer": {
            "input_dim": 400,
            "num_layers": 3,
            "output_dims": [200, 100, 5],
            "pool_sizes": 4,
            "dropout": [0.2, 0.3, 0.0]
        }
    },
    "iterator": {
        "type": "bucket",
        "sorting_keys": [["tokens", "num_tokens"]],
        "batch_size" : 4
    },
    "trainer": {
        "type": "callback",
        "num_epochs": 2,
        "optimizer": {
            "type": "adam",
            "lr": 0.001
        },
        "callbacks": [
            {"type": "track_metrics", "patience": 5},
            "validate"
        ],
        "cuda_device": 0
    }
}'''
# json.loads(config)

In [75]:
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
start_res = requests.post(url=api, json=json.loads(data))
start_res.text

'{"content": {"result": 0}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/task/edit)

In [74]:
data = '''{
    "op":"train.stop",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": {}
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
stop_res = requests.post(url=api, json=json.loads(data))
stop_res.text

'{"content": {"result": 0}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/task/edit)

### 2. 评估`/k12ai/framework/evaluate`

In [ ]:
data = '''{
    "op":"evaluate.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
data = {'type': 'k12nlp'}
res = requests.post(url=api, json=data)
res

### 3. 预测`/k12ai/framework/predict`

In [ ]:
api = 'http://%s:%d/k12ai/framework/predict' % (host, port)
data = {'type': 'k12nlp'}
res = requests.post(url=api, json=data)
res

## 平台资源管理